In [59]:
# Q1 customer gender classification
# save ROC_AUC as a csv file 
import pandas as pd
train_input = pd.read_csv('data/x_train.csv', encoding='cp949')
test_input = pd.read_csv('data/x_test.csv', encoding='cp949')
train_target = pd.read_csv('data/y_train.csv', encoding='cp949')
train_input.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85


In [60]:
train_target.head()

,cust_id,gender
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0


In [61]:
train_input.isna().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

In [62]:
train_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB


In [63]:
test_input.isna().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       1611
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

In [64]:
# drop the column of customer_id
cust_id = test_input['cust_id']

train_input = train_input.drop(columns=['cust_id']).reset_index(drop=True)
test_input = test_input.drop(columns=['cust_id']).reset_index(drop=True)
train_target = train_target.drop(columns=['cust_id']).reset_index(drop=True)

In [14]:
# replace missing values with zero as that implies that customers did not get a refund
train_input['환불금액'] = train_input['환불금액'].fillna(0)
test_input['환불금액'] = test_input['환불금액'].fillna(0)

In [16]:
# to build a classification model, the categorial variables need transforming into numbers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_input['주구매상품'])
train_input['주구매상품'] = le.transform(train_input['주구매상품'])
test_input['주구매상품'] = le.transform(test_input['주구매상품'])

le = LabelEncoder()
le.fit(train_input['주구매지점'])
train_input['주구매지점'] = le.transform(train_input['주구매지점'])
test_input['주구매지점'] = le.transform(test_input['주구매지점'])

In [17]:
train_input.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,68282840,11264000,6860000.0,5,0,19,3.894737,0.527027,17
1,2136000,2136000,300000.0,21,19,2,1.500000,0.000000,1
2,3197000,1639000,0.0,6,1,2,2.000000,0.000000,1
3,16077620,4935000,0.0,5,2,18,2.444444,0.318182,16
4,29050000,24000000,0.0,15,8,2,1.500000,0.000000,85


In [18]:
test_input.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,70900400,22000000,4050000.0,3,9,13,1.461538,0.789474,26
1,310533100,38558000,48034700.0,9,19,90,2.433333,0.369863,3
2,305264140,14825000,30521000.0,0,8,101,14.623762,0.083277,3
3,7594080,5225000,0.0,32,9,5,2.000000,0.000000,47
4,1795790,1411200,0.0,20,22,3,2.666667,0.125000,8


In [19]:
# scaling with StandardScaler
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_input = pd.DataFrame(ss.transform(train_input), columns=train_input.columns)
train_input.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,-0.144580,-0.262608,-0.047505,-0.738357,-1.904703,-0.009338,0.554247,0.758623,-0.159962
1,-0.549190,-0.547967,-0.265461,0.490702,1.466677,-0.635003,-0.698168,-1.060530,-0.806554
2,-0.542700,-0.563504,-0.275429,-0.661541,-1.727262,-0.635003,-0.436675,-1.060530,-0.806554
3,-0.463911,-0.460465,-0.275429,-0.738357,-1.549821,-0.046142,-0.204236,0.037746,-0.200374
4,-0.384561,0.135544,-0.275429,0.029805,-0.485175,-0.635003,-0.698168,-1.060530,2.588052


In [20]:
test_input = pd.DataFrame(ss.transform(test_input), columns=test_input.columns)
test_input.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,-0.128569,0.073020,-0.140867,-0.891990,-0.307733,-0.230161,-0.718283,1.664517,0.203746
1,1.337225,0.590656,1.320528,-0.431093,1.466677,2.603737,-0.210047,0.216136,-0.725730
2,1.304996,-0.151284,0.738634,-1.122438,-0.485175,3.008579,6.165384,-0.773081,-0.725730
3,-0.515803,-0.451399,-0.275429,1.335680,-0.307733,-0.524592,-0.436675,-1.060530,1.052397
4,-0.551271,-0.570626,-0.275429,0.413886,1.999000,-0.598199,-0.088017,-0.629064,-0.523670


In [23]:
train_target.values.ravel()

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [55]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(train_input, train_target.values.ravel())
print(rf.score(train_input, train_target.values.ravel()))

train_predict = rf.predict(train_input)
test_predict = rf.predict(test_input)
print(rf.score(test_input, test_predict))

0.9994285714285714
1.0


In [34]:
test_predict

array([1, 0, 0, ..., 1, 0, 1], dtype=int64)

In [57]:
from sklearn.metrics import roc_auc_score
roc_auc_score(train_target, train_predict)

0.9995421245421245

In [65]:
# store the predicted values in a file together with the customer id column
submit = pd.DataFrame({'cust_id':cust_id,'gender':test_predict})
submit.head()

,cust_id,gender
0,3500,1
1,3501,0
2,3502,0
3,3503,1
4,3504,0


In [67]:
submit.to_csv('gender_classification.csv', index=False)